# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('ETL_1',engine)
X = df.message
y = df[df.columns[4:]]

In [3]:
y = y.astype(int)

In [4]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25993 entries, 0 to 25992
Data columns (total 36 columns):
related                   25993 non-null int64
request                   25993 non-null int64
offer                     25993 non-null int64
aid_related               25993 non-null int64
medical_help              25993 non-null int64
medical_products          25993 non-null int64
search_and_rescue         25993 non-null int64
security                  25993 non-null int64
military                  25993 non-null int64
child_alone               25993 non-null int64
water                     25993 non-null int64
food                      25993 non-null int64
shelter                   25993 non-null int64
clothing                  25993 non-null int64
money                     25993 non-null int64
missing_people            25993 non-null int64
refugees                  25993 non-null int64
death                     25993 non-null int64
other_aid                 25993 non-null int6

In [5]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2.0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df['genre'].value_counts()

news      13021
direct    10615
social     2356
0             1
Name: genre, dtype: int64

In [7]:
df.drop(df[df['genre'] == 0].index, inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25993 entries, 0 to 25992
Data columns (total 40 columns):
id                        25993 non-null float64
message                   25993 non-null object
original                  25993 non-null object
genre                     25993 non-null object
related                   25993 non-null float64
request                   25993 non-null float64
offer                     25993 non-null float64
aid_related               25993 non-null float64
medical_help              25993 non-null float64
medical_products          25993 non-null float64
search_and_rescue         25993 non-null float64
security                  25993 non-null float64
military                  25993 non-null float64
child_alone               25993 non-null float64
water                     25993 non-null float64
food                      25993 non-null float64
shelter                   25993 non-null float64
clothing                  25993 non-null float64
money       

In [9]:
y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [10]:
X.isnull().sum()

0

### 2. Write a tokenization function to process your text data

In [11]:

def tokenize(text):
    url_regex ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex,text)
    for url in detected_urls:
        text = text.replace(url,"urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for token in tokens:
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_tokens)
    return text
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [14]:
X_train.shape

(19494,)

In [15]:
def evaluate_model(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    class_report = classification_report(y_test, y_pred, target_names=category_names)
    print(class_report)
    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
pred = pipeline.predict(X_test)

In [17]:
#rint(classification_report(y_test,pred))
category_names=y.columns
for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], pred[:,i], target_names=category_names), '...................................................')


related :
                        precision    recall  f1-score   support

               related       0.30      0.14      0.19      1563
               request       0.77      0.90      0.83      4936

           avg / total       0.65      0.71      0.67      6499
 ...................................................
request :
                        precision    recall  f1-score   support

               related       0.83      0.98      0.90      5362
               request       0.33      0.05      0.09      1137

           avg / total       0.74      0.82      0.76      6499
 ...................................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6472
               request       0.00      0.00      0.00        27

           avg / total       0.99      1.00      0.99      6499
 ...................................................
aid_related :
                        p

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f7f8170f268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [19]:
parameters = {
    'clf__estimator__n_estimators':[10],
    'clf__estimator__min_samples_split':[3]
}


cv = GridSearchCV(pipeline,param_grid = parameters,n_jobs=2,verbose=4)

In [29]:
print(cv)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)


In [20]:
cv.fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13727300707910126, total= 1.6min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13558017851646662, total= 1.6min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13157894736842105, total=  48.1s


[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  2.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)

In [21]:

cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.13481, std: 0.00239, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10}]

In [22]:
print(cv.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10}


In [23]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '...................................................')

related :
             precision    recall  f1-score   support

          0       0.30      0.12      0.17      1563
          1       0.77      0.91      0.83      4936

avg / total       0.65      0.72      0.67      6499
 ...................................................
request :
             precision    recall  f1-score   support

          0       0.83      0.97      0.90      5362
          1       0.33      0.06      0.10      1137

avg / total       0.74      0.81      0.76      6499
 ...................................................
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6472
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6499
 ...................................................
aid_related :
             precision    recall  f1-score   support

          0       0.59      0.76      0.66      3812
          1       0.43      0.26      0.32      26

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
with open('MLClassifier.pkl','wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.